In [1]:
import pandas as pd

In [2]:
# from google.colab import drive
# drive.mount('/content/drive')

apipath = r'H:\\My Drive\\config\\hbqa.txt'
import configparser

config = configparser.ConfigParser()
config.read(apipath)
secret_key = config['global']['OPENAI_API_KEY']
datapath = config['global']['DATA_FOLDER']
corpuspath = config['global']['CORPUS_FOLDER']

# Create Prompt for Questions Generation

In [27]:
df = pd.read_csv(datapath+'03-Chunked_Book.csv')
print(df.shape)
df.head(3)

(1963, 7)


,Section_Id,Chunk_Id,Chunk,Chunk_Letters,Chunk_Words,Chunk_Approx_Tokens,Approx_Ques
0,Book01_002,1,THE MAHABHARATA ADI PARVA Section I\nOm! Havin...,7511,1214,1615.0,4
1,Book01_002,2,The Rishi Vyasa published this mass of knowled...,7551,1261,1677.0,5
2,Book01_002,3,"Vyasa executed the compilation of the Bharata,...",6986,1162,1545.0,4


In [28]:
df.head(33)

,Section_Id,Chunk_Id,Chunk,Chunk_Letters,Chunk_Words,Chunk_Approx_Tokens,Approx_Ques
0,Book01_002,1,THE MAHABHARATA ADI PARVA Section I\nOm! Havin...,7511,1214,1615.0,4
1,Book01_002,2,The Rishi Vyasa published this mass of knowled...,7551,1261,1677.0,5
2,Book01_002,3,"Vyasa executed the compilation of the Bharata,...",6986,1162,1545.0,4
3,Book01_002,5,'When I heard that Yudhishthira had been follo...,10832,1949,2592.0,7
4,Book01_002,6,"'Alas! Gandhari, destitute of children, grand-...",7755,1246,1657.0,4
5,Book01_003,8,"Section II\n""The Rishis said, 'O son of Suta, ...",5714,974,1295.0,3
6,Book01_003,9,"'Listen, O ye ascetics, to the outlines of the...",6353,974,1295.0,3
7,Book01_003,10,The high-souled Vyasa composed these hundred p...,7600,1259,1674.0,4
8,Book01_003,11,Then comes the third parva called Aranyaka (re...,7759,1298,1726.0,5
9,Book01_003,12,"""The extensive Parva that comes next is called...",7947,1336,1777.0,5


In [4]:
len(df['Chunk'][0])

7511

In [29]:
def CreateQA_Prompt(df1, df_prompts, idx_of_files, context_id):
    for i in idx_of_files:
        Section_Id = df1.loc[i, "Section_Id"]
        Chunk_Id = df1.loc[i, "Chunk_Id"]
        Chunk_Words = df1.loc[i, "Chunk_Words"] 
        Chunk_Approx_Tokens = df1.loc[i, "Chunk_Approx_Tokens"]
        Chunk_Letters = df1.loc[i,"Chunk_Letters"]
        Questions = df1.loc[i,"Approx_Ques"]
        #question_to_generate = df1.loc[i,"Questions"]

        print ( '| Sectionid:', Section_Id,
                '| Chunk_Id:', Chunk_Id,
                "| Chunk Words:", Chunk_Words, 
                "| Chunk Approx Tokens:", Chunk_Approx_Tokens, 
                "| Chunk Letters:",Chunk_Letters,
                )
            
        text = df1.loc[i, "Chunk"]

        # prompt_ = f"""Write 1 question, corresponding answer, and their reference from the context based on the text below\n\nText: {chunk}\n\nQuestions: \n\nAnswer: \n\nReference:\n\n"""

        prompt_ = f"""Write {Questions} unique questions, corresponding answers from the context based on the text below (do not repeat the context in your response)\n\nText: {text}\n\n"""

        new_row = Section_Id, Chunk_Id, Chunk_Letters, Chunk_Words, Chunk_Approx_Tokens, prompt_ 
        df_prompts.loc[Chunk_Id] = new_row
        context_id = context_id +1

    return context_id, df_prompts


In [30]:
# Initialize dataset for prompt 
df_prompts = pd.DataFrame(columns=['Section_Id','Chunk_Id','Chunk_Letters','Chunk_Words','Chunk_Approx_Tokens', 'Prompt'])
# Initialize context_id
chunk_id = 1 

# Generate Questions & Answers for Sections.
idx_of_files = df.index
chunk_id, df_prompts = CreateQA_Prompt(df, df_prompts, idx_of_files, chunk_id)
print(f"{chunk_id} Prompt Generated")


| Sectionid: Book01_002 | Chunk_Id: 1 | Chunk Words: 1214 | Chunk Approx Tokens: 1615.0 | Chunk Letters: 7511
| Sectionid: Book01_002 | Chunk_Id: 2 | Chunk Words: 1261 | Chunk Approx Tokens: 1677.0 | Chunk Letters: 7551
| Sectionid: Book01_002 | Chunk_Id: 3 | Chunk Words: 1162 | Chunk Approx Tokens: 1545.0 | Chunk Letters: 6986
| Sectionid: Book01_002 | Chunk_Id: 5 | Chunk Words: 1949 | Chunk Approx Tokens: 2592.0 | Chunk Letters: 10832
| Sectionid: Book01_002 | Chunk_Id: 6 | Chunk Words: 1246 | Chunk Approx Tokens: 1657.0 | Chunk Letters: 7755
| Sectionid: Book01_003 | Chunk_Id: 8 | Chunk Words: 974 | Chunk Approx Tokens: 1295.0 | Chunk Letters: 5714
| Sectionid: Book01_003 | Chunk_Id: 9 | Chunk Words: 974 | Chunk Approx Tokens: 1295.0 | Chunk Letters: 6353
| Sectionid: Book01_003 | Chunk_Id: 10 | Chunk Words: 1259 | Chunk Approx Tokens: 1674.0 | Chunk Letters: 7600
| Sectionid: Book01_003 | Chunk_Id: 11 | Chunk Words: 1298 | Chunk Approx Tokens: 1726.0 | Chunk Letters: 7759
| Section

In [31]:
df_prompts.to_csv(datapath+'04-Chunk_with_Prompts.csv',index=None)

In [32]:
df = pd.read_csv(datapath+'04-Chunk_with_Prompts.csv')
df.head(33)

,Section_Id,Chunk_Id,Chunk_Letters,Chunk_Words,Chunk_Approx_Tokens,Prompt
0,Book01_002,1,7511,1214,1615.0,"Write 4 unique questions, corresponding answer..."
1,Book01_002,2,7551,1261,1677.0,"Write 5 unique questions, corresponding answer..."
2,Book01_002,3,6986,1162,1545.0,"Write 4 unique questions, corresponding answer..."
3,Book01_002,5,10832,1949,2592.0,"Write 7 unique questions, corresponding answer..."
4,Book01_002,6,7755,1246,1657.0,"Write 4 unique questions, corresponding answer..."
5,Book01_003,8,5714,974,1295.0,"Write 3 unique questions, corresponding answer..."
6,Book01_003,9,6353,974,1295.0,"Write 3 unique questions, corresponding answer..."
7,Book01_003,10,7600,1259,1674.0,"Write 4 unique questions, corresponding answer..."
8,Book01_003,11,7759,1298,1726.0,"Write 5 unique questions, corresponding answer..."
9,Book01_003,12,7947,1336,1777.0,"Write 5 unique questions, corresponding answer..."


: 